#### Experiment: Hamiltonian Pauli Weight

Using **Full SAT** or **Annealing** method.

**Full SAT**

Takes long time.

In [1]:
from itertools import product
from fermihedral import HamiltonianSolver
from fermihedral.satutil import Kissat


def calculate_model(hamiltonian_filename):
    hamiltonian_filename = "./model/" + hamiltonian_filename

    print(f"> parsing Hamiltonian from {hamiltonian_filename}")

    with open(hamiltonian_filename, "r", newline='') as hamiltonian_file:
        lines = list(map(str.strip, hamiltonian_file.readlines()))
        lines = list(filter(lambda x: len(x) > 0, lines))

    def parse_hamiltonian(lines: list[str]):
        # parse the header first
        def parse_header(line: str):
            case_name, n_modes, input_format = line.split(' ')
            return case_name, int(n_modes), input_format

        case_name, n_modes, input_format = parse_header(lines[0])

        # parse the annihilation and creation operators at each line
        occurence = []

        if input_format == "ac":
            def expand_op(op: str):
                op = abs(int(op))
                return (2 * op, 2 * op - 1)

            def delete_duplicate(item: list[int]):
                result = []
                index = 0
                while True:
                    if index >= len(item):
                        return result

                    if index == len(item) - 1:
                        result.append(item[index])
                        return result

                    if item[index + 1] == item[index]:
                        index += 2
                    else:
                        result.append(item[index])
                        index += 1

            # transform into list of creation and annihilation ops
            for line in lines[1:]:
                line = map(expand_op, line.split(" "))
                occurence.extend(product(*line))

            # filter duplicated I
            while True:
                last_length = len(occurence)
                occurence = list(filter(lambda x: len(x) > 0,
                                        map(delete_duplicate, occurence)))
                if len(occurence) == last_length:
                    break

        elif input_format == "mj":
            # directly obtain all the majoranas
            for line in lines[1:]:
                line = line.strip().split(" ")
                occurence.append(tuple(map(int, line)))

        return HamiltonianSolver(case_name, n_modes, occurence)

    solver = parse_hamiltonian(lines)

    # build a valid solution
    print(
        f"> solving with Hamiltonian Pauli weight, problem = '{solver.name}' ({solver.n_modes} modes), bk = {solver.get_bk_weight()}")

    solution, weight = solver.solve(
        progress=True, solver_init=Kissat, solver_args=[36 * 60 * 60])

    # calculate hamiltonian pauli weight under bravyi-kitaev transformation
    bk_weight = solver.get_bk_weight()

    print(
        f"{solver.name};{solver.n_modes};{weight};{bk_weight};{solution}")


for model in ["electron-4", "electron-6", "fermi-hubbard-4", "fermi-hubbard-6", "fermi-hubbard-8", "syk-3", "syk-4", "syk-5", "syk-6", "syk-7"]:
    calculate_model(model)

**Annealing**

Takes long time.

In [1]:
from itertools import product
from fermihedral import HamiltonianSolver
from fermihedral.satutil import Kissat


def anneal_model(hamiltonian_filename):
    hamiltonian_filename = "./model/" + hamiltonian_filename
    with open(hamiltonian_filename, "r", newline='') as hamiltonian_file:
        lines = list(map(str.strip, hamiltonian_file.readlines()))
        lines = list(filter(lambda x: len(x) > 0, lines))

    def parse_hamiltonian(lines: list[str]):
        # parse the header first
        def parse_header(line: str):
            case_name, n_modes, input_format = line.split(' ')
            return case_name, int(n_modes), input_format

        case_name, n_modes, input_format = parse_header(lines[0])

        # parse the annihilation and creation operators at each line
        occurence = []

        if input_format == "ac":
            def expand_op(op: str):
                op = abs(int(op))
                return (2 * op, 2 * op - 1)

            def delete_duplicate(item: list[int]):
                result = []
                index = 0
                while True:
                    if index >= len(item):
                        return result

                    if index == len(item) - 1:
                        result.append(item[index])
                        return result

                    if item[index + 1] == item[index]:
                        index += 2
                    else:
                        result.append(item[index])
                        index += 1

            # transform into list of creation and annihilation ops
            for line in lines[1:]:
                line = map(expand_op, line.split(" "))
                occurence.extend(product(*line))

            # filter duplicated I
            while True:
                last_length = len(occurence)
                occurence = list(filter(lambda x: len(x) > 0,
                                        map(delete_duplicate, occurence)))
                if len(occurence) == last_length:
                    break

        elif input_format == "mj":
            # directly obtain all the majoranas
            for line in lines[1:]:
                line = line.strip().split(" ")
                occurence.append(tuple(map(int, line)))

        return HamiltonianSolver(case_name, n_modes, occurence)

    solver = parse_hamiltonian(lines)

    # build a valid solution
    print(
        f"> solving with Hamiltonian Pauli weight, problem = '{solver.name}' ({solver.n_modes} modes), bk = {solver.get_bk_weight()}")

    solution, weight = solver.annealing(
        progress=False, initial_temp=2500, target_temp=10, alpha=10, iteration=1000)

    # calculate hamiltonian pauli weight under bravyi-kitaev transformation
    bk_weight = solver.get_bk_weight()

    print(f"{weight} (compare to {bk_weight}), improved {(bk_weight - weight) * 100 / bk_weight:.2f}%")


for model in ["electron-4", "electron-6", "electron-8", "electron-10", 
              "fermi-hubbard-4", "fermi-hubbard-6", "fermi-hubbard-8", "fermi-hubbard-10", "fermi-hubbard-12", "fermi-hubbard-14", "fermi-hubbard-16", "fermi-hubbard-18"
              "syk-3", "syk-4", "syk-5", "syk-6", "syk-7", "syk-8", "syk-9", "syk-10"]:
    anneal_model(model)

> solving with Hamiltonian Pauli weight, problem = 'electronic' (4 modes), bk = 934
> obtained solved optimal solution


100%|██████████| 249/249 [05:09<00:00,  1.24s/it]


988 (compare to 934), improved -5.78%
> solving with Hamiltonian Pauli weight, problem = 'electronic' (6 modes), bk = 9004
> obtained solved optimal solution


  6%|▌         | 15/249 [02:11<34:17,  8.79s/it]


KeyboardInterrupt: 